In [80]:
import h2o
import pandas as pd
import numpy as np
import time
from h2o.estimators.deeplearning import H2OAutoEncoderEstimator, H2ODeepLearningEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.stackedensemble  import H2OStackedEnsembleEstimator
from h2o.estimators.xgboost import H2OXGBoostEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.automl.autoh2o import H2OAutoML
from h2o.estimators.naive_bayes import H2ONaiveBayesEstimator

In [81]:
# Start H2O on your local machine
h2o.init(nthreads=-1,min_mem_size = "10g")

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 hours 49 mins
H2O cluster version:,3.14.0.2
H2O cluster version age:,4 months and 29 days !!!
H2O cluster name:,H2O_from_python_jeevan_kj9m3l
H2O cluster total nodes:,1
H2O cluster free memory:,9.20 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [95]:
#importing the datasets
train = h2o.import_file("/home/jeevan/train_1.csv")
test = h2o.import_file("/home/jeevan/test_1.csv")
submission = pd.read_csv("Sample_Solution_Yrq0zGh.csv")
print "Train dataset dimensions:", train.shape
print "Test dataset dimensions:", test.shape

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Train dataset dimensions: (69713, 29)
Test dataset dimensions: (30037, 28)


In [96]:
train.shape, test.shape

((69713, 29), (30037, 28))

In [97]:
test.describe()

Rows:30037
Cols:28




,ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,is_male,age,DOB_month,DOB_year,Lead_Creation_day,Lead_Creation_month,Lead_Creation_year
type,string,enum,time,time,enum,enum,enum,enum,int,real,enum,enum,enum,enum,enum,real,int,int,real,int,int,int,int,int,int,int,int,int
mins,NaN,,-1.1595744e+12,1.4673312e+12,,,,,1.0,0.0,,,,,,0.0,5000.0,1.0,11.99,118.0,0.0,0.0,18.0,1.0,1933.0,1.0,7.0,2016.0
mean,NaN,,4.97217475341e+11,1.47147878544e+12,,,,,3.72824783008,3977.13915171,,,,,,348.909059557,39482.9902015,3.90288482713,19.2805366763,1094.9148363,3.96231314712,0.572760262343,30.8875607645,6.59535859359,1985.25484451,16.0961813763,8.06155741252,2016.0
maxs,NaN,,9.195552e+11,1.4751936e+12,,,,,4.0,3500000.0,,,,,,43000.0,300000.0,5.0,37.0,6979.0,10.0,1.0,83.0,12.0,1999.0,31.0,9.0,2016.0
sigma,NaN,,2.23170966285e+11,2316072851.75,,,,,0.794883358458,23289.6018757,,,,,,1000.81684747,30527.8655936,1.15067296653,5.8822462436,727.452303167,3.82101963532,0.494685851134,7.08207632251,3.26688026649,7.07018583374,8.92557217925,0.819207355098,0.0
zeros,0,,0,0,,,,,0,97,,,,,,20111,0,0,0,0,9937,12833,0,0,0,0,0,0
missing,0,0,3,0,0,0,0,0,1695,0,0,0,0,0,0,32,11871,11873,20385,20385,0,0,3,3,3,0,0,0
0,APPA70109647212,Male,1988-06-03 00:00:00,2016-07-05 00:00:00,Others,C,,A,4.0,2150.0,B002,P,Y,S122,B,0.0,10000.0,3.0,20.0,372.0,4.0,1.0,28.0,6.0,1988.0,5.0,7.0,2016.0
1,APPB10687939341,Male,1981-07-13 00:00:00,2016-07-01 00:00:00,Others,A,,C,4.0,4200.0,B004,P,Y,S133,C,0.0,69000.0,5.0,24.0,1985.0,7.0,1.0,35.0,7.0,1981.0,1.0,7.0,2016.0
2,APPC80449411414,Female,1990-11-19 00:00:00,2016-07-01 00:00:00,C10009,B,,B,4.0,1000.0,B002,P,N,S133,B,0.0,nan,nan,nan,nan,0.0,0.0,26.0,11.0,1990.0,1.0,7.0,2016.0


In [98]:
train = train.drop(['ID','is_male', 'DOB_month','DOB_year','Lead_Creation_day','Lead_Creation_month','Lead_Creation_year' ])

test = test.drop(['ID','is_male', 'DOB_month','DOB_year','Lead_Creation_day','Lead_Creation_month','Lead_Creation_year' ])

In [99]:
def refine_date_col(data, col, pattern):
    #data[col]         = data[col].as_date(pattern) # As of 5/29/2106 H2O defaults parse as a date
    data["DOB_Day"]       = data[col].day()
    data["DOB_Month"]     = data[col].month()     # Since H2O indexes from 0
    data["DOB_Year"]      = data[col].year()   # Start of epoch is 1900
    data["DOB_WeekNum"]   = data[col].week()
    data["DOB_WeekDay"]   = data[col].dayOfWeek()
    
    data.describe()  # HACK: Force evaluation before ifelse and cut. See PUBDEV-1425.
    
    # Create weekend and season cols
    # Spring = Mar, Apr, May. Summer = Jun, Jul, Aug. Autumn = Sep, Oct. Winter = Nov, Dec, Jan, Feb.
    # data["Weekend"]   = [1 if x in ("Sun", "Sat") else 0 for x in data["WeekDay"]]
    data["DOB_Weekend"] = ((data["DOB_WeekDay"] == "Sun") | (data["DOB_WeekDay"] == "Sat"))
    data["DOB_Season"] = data["DOB_Month"].cut([0, 2, 5, 7, 10, 12], ["Winter", "Spring", "Summer", "Autumn", "Winter"])
    
refine_date_col(train, "DOB", "%Y-%m-%d")
train = train.drop("DOB")
train.describe()

Rows:69713
Cols:27




,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved,age,DOB_Day,DOB_Month,DOB_Year,DOB_WeekNum,DOB_WeekDay
type,enum,time,time,enum,enum,enum,enum,int,real,enum,enum,enum,enum,enum,real,int,int,real,int,int,int,int,int,int,int,int,enum
mins,,-1.2446784e+12,1.4673312e+12,,,,,1.0,0.0,,,,,,0.0,5000.0,1.0,11.99,118.0,0.0,0.0,18.0,1.0,1.0,1930.0,1.0,
mean,,4.97383139878e+11,1.47146881597e+12,,,,,3.72018650157,5622.28319969,,,,,,360.928751199,39429.9828588,3.89057924434,19.2135697612,1101.4662417,3.94844577052,0.0146314173827,30.8809004563,14.5006456426,6.59713334672,1985.26013659,26.7799793394,
maxs,,9.198144e+11,1.4751936e+12,,,,,4.0,38383838.3,,,,,,545436.5,300000.0,5.0,37.0,13556.0,10.0,1.0,86.0,31.0,12.0,1999.0,53.0,
sigma,,2.23680850544e+11,2326904996.15,,,,,0.807373964436,174767.06184,,,,,,2288.51792717,30727.5959902,1.16745935266,5.84713577597,752.661393713,3.81921382268,0.120073085331,7.09820991886,8.85670704092,3.28019400138,7.0868451027,14.3717344858,
zeros,,2,0,,,,,0,250,,,,,,46621,0,0,0,0,23308,68693,0,0,0,0,0,
missing,0,15,0,0,0,0,0,4298,0,0,0,0,0,0,51,27709,27710,47437,47437,0,0,15,15,15,15,15,15
0,Female,1979-07-23 00:00:00,2016-07-15 00:00:00,C10001,A,Others,A,4.0,2000.0,B001,P,N,S122,G,0.0,nan,nan,nan,nan,0.0,0.0,37.0,23.0,7.0,1979.0,30.0,Mon
1,Male,1986-12-07 00:00:00,2016-07-04 00:00:00,C10003,A,COM0000002,C,1.0,3500.0,B002,P,Y,S122,G,0.0,20000.0,2.0,13.25,953.0,10.0,0.0,30.0,7.0,12.0,1986.0,49.0,Sun
2,Male,1982-12-10 00:00:00,2016-07-19 00:00:00,Others,C,Others,C,4.0,2250.0,B003,G,Y,S143,B,0.0,45000.0,4.0,nan,nan,0.0,0.0,34.0,10.0,12.0,1982.0,49.0,Fri


Rows:69713
Cols:28




,Gender,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved,age,DOB_Day,DOB_Month,DOB_Year,DOB_WeekNum,DOB_WeekDay,DOB_Weekend,DOB_Season
type,enum,time,enum,enum,enum,enum,int,real,enum,enum,enum,enum,enum,real,int,int,real,int,int,int,int,int,int,int,int,enum,int,enum
mins,,1.4673312e+12,,,,,1.0,0.0,,,,,,0.0,5000.0,1.0,11.99,118.0,0.0,0.0,18.0,1.0,1.0,1930.0,1.0,,0.0,
mean,,1.47146881597e+12,,,,,3.72018650157,5622.28319969,,,,,,360.928751199,39429.9828588,3.89057924434,19.2135697612,1101.4662417,3.94844577052,0.0146314173827,30.8809004563,14.5006456426,6.59713334672,1985.26013659,26.7799793394,,0.279445727483,
maxs,,1.4751936e+12,,,,,4.0,38383838.3,,,,,,545436.5,300000.0,5.0,37.0,13556.0,10.0,1.0,86.0,31.0,12.0,1999.0,53.0,,1.0,
sigma,,2326904996.15,,,,,0.807373964436,174767.06184,,,,,,2288.51792717,30727.5959902,1.16745935266,5.84713577597,752.661393713,3.81921382268,0.120073085331,7.09820991886,8.85670704092,3.28019400138,7.0868451027,14.3717344858,,0.448730098466,
zeros,,0,,,,,0,250,,,,,,46621,0,0,0,0,23308,68693,0,0,0,0,0,,50232,
missing,0,0,0,0,0,0,4298,0,0,0,0,0,0,51,27709,27710,47437,47437,0,0,15,15,15,15,15,15,0,15
0,Female,2016-07-15 00:00:00,C10001,A,Others,A,4.0,2000.0,B001,P,N,S122,G,0.0,nan,nan,nan,nan,0.0,0.0,37.0,23.0,7.0,1979.0,30.0,Mon,0.0,Summer
1,Male,2016-07-04 00:00:00,C10003,A,COM0000002,C,1.0,3500.0,B002,P,Y,S122,G,0.0,20000.0,2.0,13.25,953.0,10.0,0.0,30.0,7.0,12.0,1986.0,49.0,Sun,1.0,Winter
2,Male,2016-07-19 00:00:00,Others,C,Others,C,4.0,2250.0,B003,G,Y,S143,B,0.0,45000.0,4.0,nan,nan,0.0,0.0,34.0,10.0,12.0,1982.0,49.0,Fri,0.0,Winter


In [100]:
refine_date_col(test, "DOB", "%Y-%m-%d")
test = test.drop("DOB")
test.describe()

Rows:30037
Cols:26




,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,age,DOB_Day,DOB_Month,DOB_Year,DOB_WeekNum,DOB_WeekDay
type,enum,time,time,enum,enum,enum,enum,int,real,enum,enum,enum,enum,enum,real,int,int,real,int,int,int,int,int,int,int,enum
mins,,-1.1595744e+12,1.4673312e+12,,,,,1.0,0.0,,,,,,0.0,5000.0,1.0,11.99,118.0,0.0,18.0,1.0,1.0,1933.0,1.0,
mean,,4.97217475341e+11,1.47147878544e+12,,,,,3.72824783008,3977.13915171,,,,,,348.909059557,39482.9902015,3.90288482713,19.2805366763,1094.9148363,3.96231314712,30.8875607645,14.571452354,6.59535859359,1985.25484451,26.7379303456,
maxs,,9.195552e+11,1.4751936e+12,,,,,4.0,3500000.0,,,,,,43000.0,300000.0,5.0,37.0,6979.0,10.0,83.0,31.0,12.0,1999.0,53.0,
sigma,,2.23170966285e+11,2316072851.75,,,,,0.794883358458,23289.6018757,,,,,,1000.81684747,30527.8655936,1.15067296653,5.8822462436,727.452303167,3.82101963532,7.08207632251,8.86878333963,3.26688026649,7.07018583374,14.3010106006,
zeros,,0,0,,,,,0,97,,,,,,20111,0,0,0,0,9937,0,0,0,0,0,
missing,0,3,0,0,0,0,0,1695,0,0,0,0,0,0,32,11871,11873,20385,20385,0,3,3,3,3,3,3
0,Male,1988-06-03 00:00:00,2016-07-05 00:00:00,Others,C,,A,4.0,2150.0,B002,P,Y,S122,B,0.0,10000.0,3.0,20.0,372.0,4.0,28.0,3.0,6.0,1988.0,22.0,Fri
1,Male,1981-07-13 00:00:00,2016-07-01 00:00:00,Others,A,,C,4.0,4200.0,B004,P,Y,S133,C,0.0,69000.0,5.0,24.0,1985.0,7.0,35.0,13.0,7.0,1981.0,29.0,Mon
2,Female,1990-11-19 00:00:00,2016-07-01 00:00:00,C10009,B,,B,4.0,1000.0,B002,P,N,S133,B,0.0,nan,nan,nan,nan,0.0,26.0,19.0,11.0,1990.0,47.0,Mon


Rows:30037
Cols:27




,Gender,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,age,DOB_Day,DOB_Month,DOB_Year,DOB_WeekNum,DOB_WeekDay,DOB_Weekend,DOB_Season
type,enum,time,enum,enum,enum,enum,int,real,enum,enum,enum,enum,enum,real,int,int,real,int,int,int,int,int,int,int,enum,int,enum
mins,,1.4673312e+12,,,,,1.0,0.0,,,,,,0.0,5000.0,1.0,11.99,118.0,0.0,18.0,1.0,1.0,1933.0,1.0,,0.0,
mean,,1.47147878544e+12,,,,,3.72824783008,3977.13915171,,,,,,348.909059557,39482.9902015,3.90288482713,19.2805366763,1094.9148363,3.96231314712,30.8875607645,14.571452354,6.59535859359,1985.25484451,26.7379303456,,0.283550287978,
maxs,,1.4751936e+12,,,,,4.0,3500000.0,,,,,,43000.0,300000.0,5.0,37.0,6979.0,10.0,83.0,31.0,12.0,1999.0,53.0,,1.0,
sigma,,2316072851.75,,,,,0.794883358458,23289.6018757,,,,,,1000.81684747,30527.8655936,1.15067296653,5.8822462436,727.452303167,3.82101963532,7.08207632251,8.86878333963,3.26688026649,7.07018583374,14.3010106006,,0.450728616465,
zeros,,0,,,,,0,97,,,,,,20111,0,0,0,0,9937,0,0,0,0,0,,21520,
missing,0,0,0,0,0,0,1695,0,0,0,0,0,0,32,11871,11873,20385,20385,0,3,3,3,3,3,3,0,3
0,Male,2016-07-05 00:00:00,Others,C,,A,4.0,2150.0,B002,P,Y,S122,B,0.0,10000.0,3.0,20.0,372.0,4.0,28.0,3.0,6.0,1988.0,22.0,Fri,0.0,Summer
1,Male,2016-07-01 00:00:00,Others,A,,C,4.0,4200.0,B004,P,Y,S133,C,0.0,69000.0,5.0,24.0,1985.0,7.0,35.0,13.0,7.0,1981.0,29.0,Mon,0.0,Summer
2,Female,2016-07-01 00:00:00,C10009,B,,B,4.0,1000.0,B002,P,N,S133,B,0.0,nan,nan,nan,nan,0.0,26.0,19.0,11.0,1990.0,47.0,Mon,0.0,Winter


In [101]:
def refine_date_col1(data, col, pattern):
    #data[col]         = data[col].as_date(pattern) # As of 5/29/2106 H2O defaults parse as a date
    data["LCD_Day"]       = data[col].day()
    data["LCD_Month"]     = data[col].month()     # Since H2O indexes from 0
    data["LCD_Year"]      = data[col].year()   # Start of epoch is 1900
    data["LCD_WeekNum"]   = data[col].week()
    data["LCD_WeekDay"]   = data[col].dayOfWeek()
    
    data.describe()  # HACK: Force evaluation before ifelse and cut. See PUBDEV-1425.
    
    # Create weekend and season cols
    # Spring = Mar, Apr, May. Summer = Jun, Jul, Aug. Autumn = Sep, Oct. Winter = Nov, Dec, Jan, Feb.
    # data["Weekend"]   = [1 if x in ("Sun", "Sat") else 0 for x in data["WeekDay"]]
    data["LCD_Weekend"] = ((data["LCD_WeekDay"] == "Sun") | (data["LCD_WeekDay"] == "Sat"))
    data["LCD_Season"] = data["LCD_Month"].cut([0, 2, 5, 7, 10, 12], ["Winter", "Spring", "Summer", "Autumn", "Winter"])
    
refine_date_col1(train, "Lead_Creation_Date", "%Y-%m-%d %H:%M:%S")
train = train.drop("Lead_Creation_Date")
train.describe()

Rows:69713
Cols:33




,Gender,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved,age,DOB_Day,DOB_Month,DOB_Year,DOB_WeekNum,DOB_WeekDay,DOB_Weekend,DOB_Season,LCD_Day,LCD_Month,LCD_Year,LCD_WeekNum,LCD_WeekDay
type,enum,time,enum,enum,enum,enum,int,real,enum,enum,enum,enum,enum,real,int,int,real,int,int,int,int,int,int,int,int,enum,int,enum,int,int,int,int,enum
mins,,1.4673312e+12,,,,,1.0,0.0,,,,,,0.0,5000.0,1.0,11.99,118.0,0.0,0.0,18.0,1.0,1.0,1930.0,1.0,,0.0,,1.0,7.0,2016.0,26.0,
mean,,1.47146881597e+12,,,,,3.72018650157,5622.28319969,,,,,,360.928751199,39429.9828588,3.89057924434,19.2135697612,1101.4662417,3.94844577052,0.0146314173827,30.8809004563,14.5006456426,6.59713334672,1985.26013659,26.7799793394,,0.279445727483,,16.0405376329,8.0596301981,2016.0,33.0245434854,
maxs,,1.4751936e+12,,,,,4.0,38383838.3,,,,,,545436.5,300000.0,5.0,37.0,13556.0,10.0,1.0,86.0,31.0,12.0,1999.0,53.0,,1.0,,31.0,9.0,2016.0,39.0,
sigma,,2326904996.15,,,,,0.807373964436,174767.06184,,,,,,2288.51792717,30727.5959902,1.16745935266,5.84713577597,752.661393713,3.81921382268,0.120073085331,7.09820991886,8.85670704092,3.28019400138,7.0868451027,14.3717344858,,0.448730098466,,8.94168577833,0.821632000176,0.0,3.85547111716,
zeros,,0,,,,,0,250,,,,,,46621,0,0,0,0,23308,68693,0,0,0,0,0,,50232,,0,0,0,0,
missing,0,0,0,0,0,0,4298,0,0,0,0,0,0,51,27709,27710,47437,47437,0,0,15,15,15,15,15,15,0,15,0,0,0,0,0
0,Female,2016-07-15 00:00:00,C10001,A,Others,A,4.0,2000.0,B001,P,N,S122,G,0.0,nan,nan,nan,nan,0.0,0.0,37.0,23.0,7.0,1979.0,30.0,Mon,0.0,Summer,15.0,7.0,2016.0,28.0,Fri
1,Male,2016-07-04 00:00:00,C10003,A,COM0000002,C,1.0,3500.0,B002,P,Y,S122,G,0.0,20000.0,2.0,13.25,953.0,10.0,0.0,30.0,7.0,12.0,1986.0,49.0,Sun,1.0,Winter,4.0,7.0,2016.0,27.0,Mon
2,Male,2016-07-19 00:00:00,Others,C,Others,C,4.0,2250.0,B003,G,Y,S143,B,0.0,45000.0,4.0,nan,nan,0.0,0.0,34.0,10.0,12.0,1982.0,49.0,Fri,0.0,Winter,19.0,7.0,2016.0,29.0,Tue


Rows:69713
Cols:34




,Gender,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved,age,DOB_Day,DOB_Month,DOB_Year,DOB_WeekNum,DOB_WeekDay,DOB_Weekend,DOB_Season,LCD_Day,LCD_Month,LCD_Year,LCD_WeekNum,LCD_WeekDay,LCD_Weekend,LCD_Season
type,enum,enum,enum,enum,enum,int,real,enum,enum,enum,enum,enum,real,int,int,real,int,int,int,int,int,int,int,int,enum,int,enum,int,int,int,int,enum,int,enum
mins,,,,,,1.0,0.0,,,,,,0.0,5000.0,1.0,11.99,118.0,0.0,0.0,18.0,1.0,1.0,1930.0,1.0,,0.0,,1.0,7.0,2016.0,26.0,,0.0,
mean,,,,,,3.72018650157,5622.28319969,,,,,,360.928751199,39429.9828588,3.89057924434,19.2135697612,1101.4662417,3.94844577052,0.0146314173827,30.8809004563,14.5006456426,6.59713334672,1985.26013659,26.7799793394,,0.279445727483,,16.0405376329,8.0596301981,2016.0,33.0245434854,,0.206977177858,
maxs,,,,,,4.0,38383838.3,,,,,,545436.5,300000.0,5.0,37.0,13556.0,10.0,1.0,86.0,31.0,12.0,1999.0,53.0,,1.0,,31.0,9.0,2016.0,39.0,,1.0,
sigma,,,,,,0.807373964436,174767.06184,,,,,,2288.51792717,30727.5959902,1.16745935266,5.84713577597,752.661393713,3.81921382268,0.120073085331,7.09820991886,8.85670704092,3.28019400138,7.0868451027,14.3717344858,,0.448730098466,,8.94168577833,0.821632000176,0.0,3.85547111716,,0.405141926014,
zeros,,,,,,0,250,,,,,,46621,0,0,0,0,23308,68693,0,0,0,0,0,,50232,,0,0,0,0,,55284,
missing,0,0,0,0,0,4298,0,0,0,0,0,0,51,27709,27710,47437,47437,0,0,15,15,15,15,15,15,0,15,0,0,0,0,0,0,0
0,Female,C10001,A,Others,A,4.0,2000.0,B001,P,N,S122,G,0.0,nan,nan,nan,nan,0.0,0.0,37.0,23.0,7.0,1979.0,30.0,Mon,0.0,Summer,15.0,7.0,2016.0,28.0,Fri,0.0,Summer
1,Male,C10003,A,COM0000002,C,1.0,3500.0,B002,P,Y,S122,G,0.0,20000.0,2.0,13.25,953.0,10.0,0.0,30.0,7.0,12.0,1986.0,49.0,Sun,1.0,Winter,4.0,7.0,2016.0,27.0,Mon,0.0,Summer
2,Male,Others,C,Others,C,4.0,2250.0,B003,G,Y,S143,B,0.0,45000.0,4.0,nan,nan,0.0,0.0,34.0,10.0,12.0,1982.0,49.0,Fri,0.0,Winter,19.0,7.0,2016.0,29.0,Tue,0.0,Summer


In [102]:
refine_date_col1(test, "Lead_Creation_Date", "%Y-%m-%d %H:%M:%S")
test = test.drop("Lead_Creation_Date")
test.describe()

Rows:30037
Cols:32




,Gender,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,age,DOB_Day,DOB_Month,DOB_Year,DOB_WeekNum,DOB_WeekDay,DOB_Weekend,DOB_Season,LCD_Day,LCD_Month,LCD_Year,LCD_WeekNum,LCD_WeekDay
type,enum,time,enum,enum,enum,enum,int,real,enum,enum,enum,enum,enum,real,int,int,real,int,int,int,int,int,int,int,enum,int,enum,int,int,int,int,enum
mins,,1.4673312e+12,,,,,1.0,0.0,,,,,,0.0,5000.0,1.0,11.99,118.0,0.0,18.0,1.0,1.0,1933.0,1.0,,0.0,,1.0,7.0,2016.0,26.0,
mean,,1.47147878544e+12,,,,,3.72824783008,3977.13915171,,,,,,348.909059557,39482.9902015,3.90288482713,19.2805366763,1094.9148363,3.96231314712,30.8875607645,14.571452354,6.59535859359,1985.25484451,26.7379303456,,0.283550287978,,16.0961813763,8.06155741252,2016.0,33.0394846356,
maxs,,1.4751936e+12,,,,,4.0,3500000.0,,,,,,43000.0,300000.0,5.0,37.0,6979.0,10.0,83.0,31.0,12.0,1999.0,53.0,,1.0,,31.0,9.0,2016.0,39.0,
sigma,,2316072851.75,,,,,0.794883358458,23289.6018757,,,,,,1000.81684747,30527.8655936,1.15067296653,5.8822462436,727.452303167,3.82101963532,7.08207632251,8.86878333963,3.26688026649,7.07018583374,14.3010106006,,0.450728616465,,8.92557217925,0.819207355098,0.0,3.83535141408,
zeros,,0,,,,,0,97,,,,,,20111,0,0,0,0,9937,0,0,0,0,0,,21520,,0,0,0,0,
missing,0,0,0,0,0,0,1695,0,0,0,0,0,0,32,11871,11873,20385,20385,0,3,3,3,3,3,3,0,3,0,0,0,0,0
0,Male,2016-07-05 00:00:00,Others,C,,A,4.0,2150.0,B002,P,Y,S122,B,0.0,10000.0,3.0,20.0,372.0,4.0,28.0,3.0,6.0,1988.0,22.0,Fri,0.0,Summer,5.0,7.0,2016.0,27.0,Tue
1,Male,2016-07-01 00:00:00,Others,A,,C,4.0,4200.0,B004,P,Y,S133,C,0.0,69000.0,5.0,24.0,1985.0,7.0,35.0,13.0,7.0,1981.0,29.0,Mon,0.0,Summer,1.0,7.0,2016.0,26.0,Fri
2,Female,2016-07-01 00:00:00,C10009,B,,B,4.0,1000.0,B002,P,N,S133,B,0.0,nan,nan,nan,nan,0.0,26.0,19.0,11.0,1990.0,47.0,Mon,0.0,Winter,1.0,7.0,2016.0,26.0,Fri


Rows:30037
Cols:33




,Gender,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,age,DOB_Day,DOB_Month,DOB_Year,DOB_WeekNum,DOB_WeekDay,DOB_Weekend,DOB_Season,LCD_Day,LCD_Month,LCD_Year,LCD_WeekNum,LCD_WeekDay,LCD_Weekend,LCD_Season
type,enum,enum,enum,enum,enum,int,real,enum,enum,enum,enum,enum,real,int,int,real,int,int,int,int,int,int,int,enum,int,enum,int,int,int,int,enum,int,enum
mins,,,,,,1.0,0.0,,,,,,0.0,5000.0,1.0,11.99,118.0,0.0,18.0,1.0,1.0,1933.0,1.0,,0.0,,1.0,7.0,2016.0,26.0,,0.0,
mean,,,,,,3.72824783008,3977.13915171,,,,,,348.909059557,39482.9902015,3.90288482713,19.2805366763,1094.9148363,3.96231314712,30.8875607645,14.571452354,6.59535859359,1985.25484451,26.7379303456,,0.283550287978,,16.0961813763,8.06155741252,2016.0,33.0394846356,,0.208675966308,
maxs,,,,,,4.0,3500000.0,,,,,,43000.0,300000.0,5.0,37.0,6979.0,10.0,83.0,31.0,12.0,1999.0,53.0,,1.0,,31.0,9.0,2016.0,39.0,,1.0,
sigma,,,,,,0.794883358458,23289.6018757,,,,,,1000.81684747,30527.8655936,1.15067296653,5.8822462436,727.452303167,3.82101963532,7.08207632251,8.86878333963,3.26688026649,7.07018583374,14.3010106006,,0.450728616465,,8.92557217925,0.819207355098,0.0,3.83535141408,,0.406369050421,
zeros,,,,,,0,97,,,,,,20111,0,0,0,0,9937,0,0,0,0,0,,21520,,0,0,0,0,,23769,
missing,0,0,0,0,0,1695,0,0,0,0,0,0,32,11871,11873,20385,20385,0,3,3,3,3,3,3,0,3,0,0,0,0,0,0,0
0,Male,Others,C,,A,4.0,2150.0,B002,P,Y,S122,B,0.0,10000.0,3.0,20.0,372.0,4.0,28.0,3.0,6.0,1988.0,22.0,Fri,0.0,Summer,5.0,7.0,2016.0,27.0,Tue,0.0,Summer
1,Male,Others,A,,C,4.0,4200.0,B004,P,Y,S133,C,0.0,69000.0,5.0,24.0,1985.0,7.0,35.0,13.0,7.0,1981.0,29.0,Mon,0.0,Summer,1.0,7.0,2016.0,26.0,Fri,0.0,Summer
2,Female,C10009,B,,B,4.0,1000.0,B002,P,N,S133,B,0.0,nan,nan,nan,nan,0.0,26.0,19.0,11.0,1990.0,47.0,Mon,0.0,Winter,1.0,7.0,2016.0,26.0,Fri,0.0,Summer


In [115]:
test[test['Employer_Code']=='', :]

In [116]:
test.describe()

Rows:30037
Cols:39




,Gender,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,age,DOB_Day,DOB_Month,DOB_Year,DOB_WeekNum,DOB_WeekDay,DOB_Weekend,DOB_Season,LCD_Day,LCD_Month,LCD_Year,LCD_WeekNum,LCD_WeekDay,LCD_Weekend,LCD_Season,Employer_Category2Missing,Existing_EMIMissing,Loan_AmountMissing,Loan_PeriodMissing,Interest_RateMissing,EMIMissing
type,enum,enum,enum,enum,enum,enum,real,enum,enum,enum,enum,enum,real,int,enum,real,int,enum,int,int,int,int,int,enum,int,enum,int,int,int,int,enum,int,enum,int,int,int,int,int,int
mins,,,,,,,0.0,,,,,,0.0,5000.0,,11.99,118.0,,18.0,1.0,1.0,1933.0,1.0,,0.0,,1.0,7.0,2016.0,26.0,,0.0,,0.0,0.0,0.0,0.0,0.0,0.0
mean,,,,,,,3977.13915171,,,,,,348.909059557,39482.9902015,,19.2805366763,1094.9148363,,30.8875607645,14.571452354,6.59535859359,1985.25484451,26.7379303456,,0.283550287978,,16.0961813763,8.06155741252,2016.0,33.0394846356,,0.208675966308,,0.943569597496,0.998934647268,0.604787428838,0.604720844292,0.321337017678,0.321337017678
maxs,,,,,,,3500000.0,,,,,,43000.0,300000.0,,37.0,6979.0,,83.0,31.0,12.0,1999.0,53.0,,1.0,,31.0,9.0,2016.0,39.0,,1.0,,1.0,1.0,1.0,1.0,1.0,1.0
sigma,,,,,,,23289.6018757,,,,,,1000.81684747,30527.8655936,,5.8822462436,727.452303167,,7.08207632251,8.86878333963,3.26688026649,7.07018583374,14.3010106006,,0.450728616465,,8.92557217925,0.819207355098,0.0,3.83535141408,,0.406369050421,,0.230754815587,0.0326228935963,0.488904441101,0.488918707973,0.466997643841,0.466997643841
zeros,,,,,,,97,,,,,,20111,0,,0,0,,0,0,0,0,0,,21520,,0,0,0,0,,23769,,1695,32,11871,11873,20385,20385
missing,0,0,0,26827,0,1695,0,0,0,0,0,0,32,11871,11873,20385,20385,0,3,3,3,3,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Male,Others,C,,A,4,2150.0,B002,P,Y,S122,B,0.0,10000.0,3,20.0,372.0,4,28.0,3.0,6.0,1988.0,22.0,Fri,0.0,Summer,5.0,7.0,2016.0,27.0,Tue,0.0,Summer,1.0,1.0,1.0,1.0,1.0,1.0
1,Male,Others,A,,C,4,4200.0,B004,P,Y,S133,C,0.0,69000.0,5,24.0,1985.0,7,35.0,13.0,7.0,1981.0,29.0,Mon,0.0,Summer,1.0,7.0,2016.0,26.0,Fri,0.0,Summer,1.0,1.0,1.0,1.0,1.0,1.0
2,Female,C10009,B,,B,4,1000.0,B002,P,N,S133,B,0.0,nan,,nan,nan,0,26.0,19.0,11.0,1990.0,47.0,Mon,0.0,Winter,1.0,7.0,2016.0,26.0,Fri,0.0,Summer,1.0,1.0,0.0,0.0,0.0,0.0


In [105]:
train[69700,:]

Gender,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved,age,DOB_Day,DOB_Month,DOB_Year,DOB_WeekNum,DOB_WeekDay,DOB_Weekend,DOB_Season,LCD_Day,LCD_Month,LCD_Year,LCD_WeekNum,LCD_WeekDay,LCD_Weekend,LCD_Season
Female,Others,C,Others,A,4,5152.4,B001,P,N,S122,G,2364.8,nan,nan,nan,nan,0,0,53,2,1,1964,1,Thu,0,Winter,30,9,2016,39,Fri,0,Autumn


In [106]:
# Convert 'Survived' and 'Pclass' to categorical values
train['Approved'] = train['Approved'].asfactor()
train['Employer_Category2'] = train['Employer_Category2'].asfactor()
train['Loan_Period'] = train['Loan_Period'].asfactor()
train['Var1'] = train['Var1'].asfactor()

In [107]:
test['Employer_Category2'] = test['Employer_Category2'].asfactor()
test['Loan_Period'] = test['Loan_Period'].asfactor()
test['Var1'] = test['Var1'].asfactor()

In [108]:
test['Employer_Code'].table()

Employer_Code,Count
COM0000002,165
COM0000003,124
COM0000004,130
COM0000005,81
COM0000006,64
COM0000007,78
COM0000008,69
COM0000009,56
COM0000010,53
COM0000011,60


In [109]:
#training feature engineering
train['Employer_Category2Missing'] = (train["Employer_Category2"].isna()).ifelse(0, 1)
train['Existing_EMIMissing'] = (train["Existing_EMI"].isna()).ifelse(0, 1)
train['Loan_AmountMissing'] = (train["Loan_Amount"].isna()).ifelse(0, 1)
train['Loan_PeriodMissing'] = (train["Loan_Period"].isna()).ifelse(0, 1)
train['Interest_RateMissing'] = (train["Interest_Rate"].isna()).ifelse(0, 1)
train['EMIMissing'] = (train["EMI"].isna()).ifelse(0, 1)

In [110]:
#testing feature engineering
test['Employer_Category2Missing'] = (test["Employer_Category2"].isna()).ifelse(0, 1)
test['Existing_EMIMissing'] = (test["Existing_EMI"].isna()).ifelse(0, 1)
test['Loan_AmountMissing'] = (test["Loan_Amount"].isna()).ifelse(0, 1)
test['Loan_PeriodMissing'] = (test["Loan_Period"].isna()).ifelse(0, 1)
test['Interest_RateMissing'] = (test["Interest_Rate"].isna()).ifelse(0, 1)
test['EMIMissing'] = (test["EMI"].isna()).ifelse(0, 1)

In [111]:
train.describe()

Rows:69713
Cols:40




,Gender,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved,age,DOB_Day,DOB_Month,DOB_Year,DOB_WeekNum,DOB_WeekDay,DOB_Weekend,DOB_Season,LCD_Day,LCD_Month,LCD_Year,LCD_WeekNum,LCD_WeekDay,LCD_Weekend,LCD_Season,Employer_Category2Missing,Existing_EMIMissing,Loan_AmountMissing,Loan_PeriodMissing,Interest_RateMissing,EMIMissing
type,enum,enum,enum,enum,enum,enum,real,enum,enum,enum,enum,enum,real,int,enum,real,int,enum,enum,int,int,int,int,int,enum,int,enum,int,int,int,int,enum,int,enum,int,int,int,int,int,int
mins,,,,,,,0.0,,,,,,0.0,5000.0,,11.99,118.0,,,18.0,1.0,1.0,1930.0,1.0,,0.0,,1.0,7.0,2016.0,26.0,,0.0,,0.0,0.0,0.0,0.0,0.0,0.0
mean,,,,,,,5622.28319969,,,,,,360.928751199,39429.9828588,,19.2135697612,1101.4662417,,,30.8809004563,14.5006456426,6.59713334672,1985.26013659,26.7799793394,,0.279445727483,,16.0405376329,8.0596301981,2016.0,33.0245434854,,0.206977177858,,0.938347223617,0.999268429131,0.60252750563,0.602513161103,0.319538680017,0.319538680017
maxs,,,,,,,38383838.3,,,,,,545436.5,300000.0,,37.0,13556.0,,,86.0,31.0,12.0,1999.0,53.0,,1.0,,31.0,9.0,2016.0,39.0,,1.0,,1.0,1.0,1.0,1.0,1.0,1.0
sigma,,,,,,,174767.06184,,,,,,2288.51792717,30727.5959902,,5.84713577597,752.661393713,,,7.09820991886,8.85670704092,3.28019400138,7.0868451027,14.3717344858,,0.448730098466,,8.94168577833,0.821632000176,0.0,3.85547111716,,0.405141926014,,0.240525552519,0.027037865295,0.489378734706,0.489381739786,0.466301223479,0.466301223479
zeros,,,,,,,250,,,,,,46621,0,,0,0,,,0,0,0,0,0,,50232,,0,0,0,0,,55284,,4298,51,27709,27710,47437,47437
missing,0,0,0,0,0,4298,0,0,0,0,0,0,51,27709,27710,47437,47437,0,0,15,15,15,15,15,15,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Female,C10001,A,Others,A,4,2000.0,B001,P,N,S122,G,0.0,nan,,nan,nan,0,0,37.0,23.0,7.0,1979.0,30.0,Mon,0.0,Summer,15.0,7.0,2016.0,28.0,Fri,0.0,Summer,1.0,1.0,0.0,0.0,0.0,0.0
1,Male,C10003,A,COM0000002,C,1,3500.0,B002,P,Y,S122,G,0.0,20000.0,2,13.25,953.0,10,0,30.0,7.0,12.0,1986.0,49.0,Sun,1.0,Winter,4.0,7.0,2016.0,27.0,Mon,0.0,Summer,1.0,1.0,1.0,1.0,1.0,1.0
2,Male,Others,C,Others,C,4,2250.0,B003,G,Y,S143,B,0.0,45000.0,4,nan,nan,0,0,34.0,10.0,12.0,1982.0,49.0,Fri,0.0,Winter,19.0,7.0,2016.0,29.0,Tue,0.0,Summer,1.0,1.0,1.0,1.0,0.0,0.0


In [112]:
test.describe()

Rows:30037
Cols:39




,Gender,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,age,DOB_Day,DOB_Month,DOB_Year,DOB_WeekNum,DOB_WeekDay,DOB_Weekend,DOB_Season,LCD_Day,LCD_Month,LCD_Year,LCD_WeekNum,LCD_WeekDay,LCD_Weekend,LCD_Season,Employer_Category2Missing,Existing_EMIMissing,Loan_AmountMissing,Loan_PeriodMissing,Interest_RateMissing,EMIMissing
type,enum,enum,enum,enum,enum,enum,real,enum,enum,enum,enum,enum,real,int,enum,real,int,enum,int,int,int,int,int,enum,int,enum,int,int,int,int,enum,int,enum,int,int,int,int,int,int
mins,,,,,,,0.0,,,,,,0.0,5000.0,,11.99,118.0,,18.0,1.0,1.0,1933.0,1.0,,0.0,,1.0,7.0,2016.0,26.0,,0.0,,0.0,0.0,0.0,0.0,0.0,0.0
mean,,,,,,,3977.13915171,,,,,,348.909059557,39482.9902015,,19.2805366763,1094.9148363,,30.8875607645,14.571452354,6.59535859359,1985.25484451,26.7379303456,,0.283550287978,,16.0961813763,8.06155741252,2016.0,33.0394846356,,0.208675966308,,0.943569597496,0.998934647268,0.604787428838,0.604720844292,0.321337017678,0.321337017678
maxs,,,,,,,3500000.0,,,,,,43000.0,300000.0,,37.0,6979.0,,83.0,31.0,12.0,1999.0,53.0,,1.0,,31.0,9.0,2016.0,39.0,,1.0,,1.0,1.0,1.0,1.0,1.0,1.0
sigma,,,,,,,23289.6018757,,,,,,1000.81684747,30527.8655936,,5.8822462436,727.452303167,,7.08207632251,8.86878333963,3.26688026649,7.07018583374,14.3010106006,,0.450728616465,,8.92557217925,0.819207355098,0.0,3.83535141408,,0.406369050421,,0.230754815587,0.0326228935963,0.488904441101,0.488918707973,0.466997643841,0.466997643841
zeros,,,,,,,97,,,,,,20111,0,,0,0,,0,0,0,0,0,,21520,,0,0,0,0,,23769,,1695,32,11871,11873,20385,20385
missing,0,0,0,26827,0,1695,0,0,0,0,0,0,32,11871,11873,20385,20385,0,3,3,3,3,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Male,Others,C,,A,4,2150.0,B002,P,Y,S122,B,0.0,10000.0,3,20.0,372.0,4,28.0,3.0,6.0,1988.0,22.0,Fri,0.0,Summer,5.0,7.0,2016.0,27.0,Tue,0.0,Summer,1.0,1.0,1.0,1.0,1.0,1.0
1,Male,Others,A,,C,4,4200.0,B004,P,Y,S133,C,0.0,69000.0,5,24.0,1985.0,7,35.0,13.0,7.0,1981.0,29.0,Mon,0.0,Summer,1.0,7.0,2016.0,26.0,Fri,0.0,Summer,1.0,1.0,1.0,1.0,1.0,1.0
2,Female,C10009,B,,B,4,1000.0,B002,P,N,S133,B,0.0,nan,,nan,nan,0,26.0,19.0,11.0,1990.0,47.0,Mon,0.0,Winter,1.0,7.0,2016.0,26.0,Fri,0.0,Summer,1.0,1.0,0.0,0.0,0.0,0.0


### Defining Search Criteria

In [117]:
# define the criteria for random grid search
search_criteria = {'strategy': "RandomDiscrete", 
                   'max_models': 9,
                   'seed': 1234}

In [118]:
# Number of CV folds (to generate level-one data for stacking)
nfolds = 5

In [119]:
features = train.names
features.remove("Approved")

In [120]:
features

[u'Gender',
 u'City_Code',
 u'City_Category',
 u'Employer_Code',
 u'Employer_Category1',
 u'Employer_Category2',
 u'Monthly_Income',
 u'Customer_Existing_Primary_Bank_Code',
 u'Primary_Bank_Type',
 u'Contacted',
 u'Source',
 u'Source_Category',
 u'Existing_EMI',
 u'Loan_Amount',
 u'Loan_Period',
 u'Interest_Rate',
 u'EMI',
 u'Var1',
 u'age',
 u'DOB_Day',
 u'DOB_Month',
 u'DOB_Year',
 u'DOB_WeekNum',
 u'DOB_WeekDay',
 u'DOB_Weekend',
 u'DOB_Season',
 u'LCD_Day',
 u'LCD_Month',
 u'LCD_Year',
 u'LCD_WeekNum',
 u'LCD_WeekDay',
 u'LCD_Weekend',
 u'LCD_Season',
 u'Employer_Category2Missing',
 u'Existing_EMIMissing',
 u'Loan_AmountMissing',
 u'Loan_PeriodMissing',
 u'Interest_RateMissing',
 'EMIMissing']

## GBM

In [121]:
# define the range of hyper-parameters for GBM grid search
# 27 combinations in total
hyper_params = {'sample_rate': [0.7, 0.8, 0.9],
                'col_sample_rate': [0.7, 0.8, 0.9],
                'max_depth': [3, 5, 7]}

In [122]:
# Set up GBM grid search
# Add a seed for reproducibility
gbm_rand_grid = H2OGridSearch(
                    H2OGradientBoostingEstimator(
                        model_id = 'gbm_rand_grid', 
                        seed = 1234,
                        ntrees = 10000,   
                        nfolds = nfolds,
                        fold_assignment = "Modulo",               # needed for stacked ensembles
                        keep_cross_validation_predictions = True, # needed for stacked ensembles
                        stopping_metric = 'mse', 
                        stopping_rounds = 15,     
                        score_tree_interval = 1),
                    search_criteria = search_criteria, # full grid search
                    hyper_params = hyper_params)

In [123]:
# Use .train() to start the grid search
gbm_rand_grid.train(x = features, 
                    y = 'Approved', 
                    training_frame = train)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [131]:
gbm_rand_grid_sorted = gbm_rand_grid.get_grid(sort_by='auc', decreasing=False)
print(gbm_rand_grid_sorted)

    col_sample_rate max_depth sample_rate  \
0               0.8         7         0.7   
1               0.9         7         0.7   
2               0.7         7         0.7   
3               0.9         7         0.9   
4               0.8         3         0.9   
5               0.7         5         0.8   
6               0.7         3         0.7   
7               0.8         3         0.8   
8               0.9         3         0.9   

                                                           model_ids  \
0  Grid_GBM_py_242_sid_b199_model_python_1516480334597_16119_model_4   
1  Grid_GBM_py_242_sid_b199_model_python_1516480334597_16119_model_6   
2  Grid_GBM_py_242_sid_b199_model_python_1516480334597_16119_model_1   
3  Grid_GBM_py_242_sid_b199_model_python_1516480334597_16119_model_5   
4  Grid_GBM_py_242_sid_b199_model_python_1516480334597_16119_model_7   
5  Grid_GBM_py_242_sid_b199_model_python_1516480334597_16119_model_0   
6  Grid_GBM_py_242_sid_b199_model_python_1516

In [132]:
best_model_id = gbm_rand_grid_sorted.model_ids[0]
best_gbm_from_rand_grid = h2o.get_model(best_model_id)
best_gbm_from_rand_grid.summary()

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,36.0,36.0,51973.0,7.0,7.0,7.0,55.0,96.0,74.44444


In [133]:
preds = best_gbm_from_rand_grid.predict(test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [134]:
preds=preds['p1']

In [135]:
test_ids = pd.read_csv("test_ids.csv", names = ['ID'])

In [136]:
test_ids_hf = h2o.H2OFrame(test_ids)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [137]:
submission = pd.concat((h2o.as_list(test_ids_hf['ID']), h2o.as_list(preds)), axis=1, ignore_index=True)
submission.columns = ['ID', 'Approved']
submission = submission.set_index(['ID', 'Approved'])
submission.to_csv('submission_h2o_gbm.csv')